In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
from sklearn.decomposition import PCA

In [6]:
processed = pd.read_csv('./data/course_processed.csv')
bows_df = pd.read_csv('./data/courses_bows.csv')
bows_df = bows_df[['doc_id','token','bow']]
bows_df

,doc_id,token,bow
0,ML0201EN,ai,2
1,ML0201EN,apps,2
2,ML0201EN,build,2
3,ML0201EN,cloud,1
4,ML0201EN,coming,1
...,...,...,...
10361,excourse93,modifying,1
10362,excourse93,objectives,1
10363,excourse93,pieces,1
10364,excourse93,plugins,1


In [4]:
def pivot_two_bows(basedoc, comparedoc):
    base = basedoc.copy()
    base['type'] = 'base'
    compare = comparedoc.copy()
    compare['type'] = 'compare'

    merged = pd.concat([base, compare], ignore_index=True)

    pivot_table = merged.pivot_table(index=['doc_id', 'type'], columns='token', fill_value=0)

    pivot_table = pivot_table.reset_index()

    pivot_table.columns = pivot_table.columns.droplevel(0)

    pivot_table = pivot_table.rename_axis(columns=None)
    pivot_table[pivot_table.columns[2:]] = pivot_table.iloc[:,2:].astype('float32')
    return pivot_table

In [5]:
course1 = bows_df[bows_df['doc_id'] == 'ML0151EN']
course2 = bows_df[bows_df['doc_id'] == 'ML0101ENv3']

bow_vectors = pivot_two_bows(course1, course2)
bow_vectors

,,,approachable,basics,beneficial,comparison,course,dives,free,future,...,relates,started,statistical,supervised,tool,tools,trends,unsupervised,using,vs
0,ML0101ENv3,compare,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1,ML0151EN,base,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0


In [ ]:
s = []
for j in processed.iloc[:,0]:
    sims={}
    for i in processed.iloc[:100,0]:
        base = bows_df[bows_df['doc_id'] == j]
        comparedoc = bows_df[bows_df['doc_id'] == i]
        bow_vectors = pivot_two_bows(base, comparedoc)
        similarity = 1 - cosine(bow_vectors.iloc[0, 2:], bow_vectors.iloc[1, 2:])
        sims[i]=similarity
    s.append(sims)

In [8]:
sim = pd.DataFrame(s)

NameError: name 's' is not defined

In [ ]:
sim.to_csv('./data/similarity.csv', index=False)